# Übung 19 - RNN_LSTM

## RNN

In [ ]:
# notwendige libraries laden
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [ ]:
# 1. Daten laden und vorbereiten
## Der IMDB-Datensatz enthält Rezessionen zu Filmen, welche als positiv (1) oder negativ (0) gelabelt sind
## Im Datensatz (x, y) sind nicht direkt die Wörter enthalten, sondern Wortindizes mit Verweis auf einen Dictionary
## Beim Abrufen des Datensatzes, kann zudem angegeben werden, wie viele der häufigsten Wörter berücksichtigt werden sollen

max_features = 10000  # Anzahl der häufigsten Wörter, die berücksichtigt werden
maxlen = 500  # Maximale Länge der Filmbewertungen

print('Lade Daten...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), 'Trainingssequenzen')
print(len(input_test), 'Testsequenzen')

print('Sequenzen mit Padding versehen...')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

Lade Daten...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
25000 Trainingssequenzen
25000 Testsequenzen
Sequenzen mit Padding versehen...
input_train shape: (25000, 500)
input_test shape: (25000, 500)


In [ ]:
# Wörter aus den Wortindizes abrufen
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
print(reverse_word_index[1])

the


In [ ]:
# 2. Modell erstellen
## ein einfaches RNN mit nur einem RNN-Layer
## Als erstes müssen aber die Word-Embeddings erstellt werden (Wort --> Vektor mit 32 Zahlen)
## Die RNN Schicht erhält 32 Neuronen
## Für die Klassifikation wird ein einzelnes Neuron mit Sigmoid-Aktivierung genommen
model = keras.Sequential()
model.add(keras.layers.Embedding(max_features, 32))
model.add(keras.layers.SimpleRNN(32))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
# 3. Modell kompilieren
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
# 4. Modell trainieren
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 27s 148ms/step - acc: 0.5858 - loss: 0.6622 - val_acc: 0.6572 - val_loss: 0.6885
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 27s 78ms/step - acc: 0.8050 - loss: 0.4507 - val_acc: 0.7722 - val_loss: 0.4818
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 15s 98ms/step - acc: 0.8629 - loss: 0.3367 - val_acc: 0.7852 - val_loss: 0.4474
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - acc: 0.8933 - loss: 0.2745 - val_acc: 0.8412 - val_loss: 0.3937
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 73ms/step - acc: 0.9090 - loss: 0.2347 - val_acc: 0.7930 - val_loss: 0.5277
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 69ms/step - acc: 0.9350 - loss: 0.1801 - val_acc: 0.7628 - val_loss: 0.4828
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - acc: 0.9508 - loss: 0.1443 - val_acc: 0.8560 - val_loss: 0.4101
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 21s 72ms/step - acc: 0.9703 - loss: 0.0909 - val_acc: 0.8280 - val_loss: 0.5174
Epoch 9/10
157/157 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 5. Modell evaluieren
results = model.evaluate(input_test, y_test)
print(results)

782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - acc: 0.8476 - loss: 0.5437
[0.5384258031845093, 0.8487600088119507]


In [ ]:
# Modell nutzen für eine Vorhersage

new_review = "This movie was absolute crap! I hate it from the very first minute..."

# Die Bewertung muss auch in die Wortindizes umgewandelt werden
word_index = imdb.get_word_index()
new_review_indices = [word_index.get(word, 0) + 3 for word in new_review.split()]  # 3 addieren für Padding, Start-of-Sequence, Unknown
new_review_indices = sequence.pad_sequences([new_review_indices], maxlen=maxlen)  # maxlen ist die maximale Sequenzlänge aus dem Training

# Bewertung durchführen
prediction = model.predict(new_review_indices)

# Ergebnis interpretieren
if prediction[0][0] > 0.5:
    print("Positive Bewertung")
else:
    print("Negative Bewertung")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Positive Bewertung


## LSTM

In [ ]:
# notwendige libraries laden
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [ ]:
# 1. Daten laden und vorbereiten
## Der IMDB-Datensatz enthält Rezessionen zu Filmen, welche als positiv (1) oder negativ (0) gelabelt sind
## Im Datensatz (x, y) sind nicht direkt die Wörter enthalten, sondern Wortindizes mit Verweis auf einen Dictionary
## Beim Abrufen des Datensatzes, kann zudem angegeben werden, wie viele der häufigsten Wörter berücksichtig werden sollen

max_features = 10000  # Anzahl der häufigsten Wörter, die berücksichtigt werden
maxlen = 500  # Maximale Länge der Filmbewertungen

print('Lade Daten...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), 'Trainingssequenzen')
print(len(input_test), 'Testsequenzen')

print('Sequenzen mit Padding versehen...')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

Lade Daten...
25000 Trainingssequenzen
25000 Testsequenzen
Sequenzen mit Padding versehen...
input_train shape: (25000, 500)
input_test shape: (25000, 500)


In [ ]:
# Wörter aus den Wortindizes abrufen
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
print(reverse_word_index[1])

the


In [ ]:
# 2. Modell erstellen
model = keras.Sequential()
model.add(keras.layers.Embedding(max_features, 32))
model.add(keras.layers.LSTM(32))  # LSTM-Schicht hinzugefügt
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
# 3. Modell kompilieren
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
# 4. Modell trainieren
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - acc: 0.5755 - loss: 0.6659 - val_acc: 0.7896 - val_loss: 0.4589
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - acc: 0.8124 - loss: 0.4261 - val_acc: 0.6932 - val_loss: 0.7660
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - acc: 0.8579 - loss: 0.3538 - val_acc: 0.8740 - val_loss: 0.3135
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - acc: 0.8748 - loss: 0.3040 - val_acc: 0.8416 - val_loss: 0.4003
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - acc: 0.8895 - loss: 0.2777 - val_acc: 0.8722 - val_loss: 0.3062
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - acc: 0.9055 - loss: 0.2472 - val_acc: 0.8782 - val_loss: 0.2979
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - acc: 0.9098 - loss: 0.2327 - val_acc: 0.8764 - val_loss: 0.2940
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - acc: 0.9113 - loss: 0.2245 - val_acc: 0.8600 - val_loss: 0.3311
Epoch 9/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/

In [ ]:
# 5. Modell evaluieren
results = model.evaluate(input_test, y_test)
print(results)

782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - acc: 0.8679 - loss: 0.3583
[0.3493182063102722, 0.8698400259017944]


In [ ]:
# Modell nutzen für eine Vorhersage

new_review = "This movie was absolute crap! I hate it from the very first minute..."

# Die Bewertung muss auch in die Wortindizes umgewandelt werden
word_index = imdb.get_word_index()
new_review_indices = [word_index.get(word, 0) + 3 for word in new_review.split()]  # 3 addieren für Padding, Start-of-Sequence, Unknown
new_review_indices = sequence.pad_sequences([new_review_indices], maxlen=maxlen)  # maxlen ist die maximale Sequenzlänge aus dem Training

# Bewertung durchführen
prediction = model.predict(new_review_indices)

# Ergebnis interpretieren
if prediction[0][0] > 0.5:
    print("Positive Bewertung")
else:
    print("Negative Bewertung")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Positive Bewertung
